In [1]:
import os

In [2]:
%pwd

'e:\\Diabetes_classifier\\research'

In [3]:
os.chdir('../')

In [5]:
from  pathlib import Path
from dataclasses import dataclass


@dataclass
class Model_trainerConfig:
    root_dir :Path
    model_dir :Path
    
    


In [6]:
from src.DIABETES_CLASSIFIER.utils.common import read_yaml,create_directories
from src.DIABETES_CLASSIFIER.constants import config_file_path,param_file_path
from src.DIABETES_CLASSIFIER import logger

In [7]:
class Configuratoration:
    def __init__(self,config=config_file_path,params=param_file_path):
        self.config=read_yaml(config)
        self.params=read_yaml(params)

        create_directories([self.config.artifacts_root])

    def get_model_trainer(self)->Model_trainerConfig:
        
        data=self.config.data_ingestion
        trainer=self.config.data_trainer
        transformation=self.config.data_transformation


        create_directories([trainer.root_dir])

        trainer_model=Model_trainerConfig(
            root_dir=trainer.root_dir,
            model_dir=trainer.model_dir
        )



        return (data,transformation,trainer_model)

In [8]:
from src.DIABETES_CLASSIFIER.components.data_transformation import Data_Transformation
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from catboost import CatBoostClassifier
from xgboost import XGBRFClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,recall_score
from sklearn.neighbors import KNeighborsClassifier
from src.DIABETES_CLASSIFIER.utils.common import  model_prediction,save_model


In [12]:
class Model_trainer:
    def __init__(self,config,data,transformation):
        self.config=config
        self.data=data
        self.transformation=transformation
 

    def get_trained_model(self,x_train,x_test,y_train,y_test):
        models={
            'logistic_model':LogisticRegression(),
            'naive_bayes_model':GaussianNB(),
            'svc':SVC(),
            'Decision_tree':DecisionTreeClassifier(),
            'Kn_neighbours':KNeighborsClassifier(),
            'Random_forest':RandomForestClassifier(),
            'Gradient_boosting':GradientBoostingClassifier(),
            'Cat_boosting':CatBoostClassifier(),
            'Xg_boosting':XGBRFClassifier()}
        

        
        model_list,r_score_list,score_list=model_prediction(models=models,x_train=x_train,x_test=x_test,y_train=y_train,y_test=y_test)
        
        model_report={}

        for x,y in zip(model_list,r_score_list):
            model_report[x]=y

        high_values_models=sorted(model_report.items(),key=lambda x:x[1],reverse=True)

        print(high_values_models)

        best_model=models[high_values_models[0][0]]


        save_model(obj=best_model,file_path=self.config.model_dir)


In [33]:
from src.DIABETES_CLASSIFIER.components.data_transformation import Data_Transformation


In [14]:
try:
    config=Configuratoration()
    data,transformation,trainer_model=config.get_model_trainer()

    transformation=Data_Transformation(config=transformation,ingestion=data)

    x_train,x_test,y_train,y_test=transformation.data_tranformation_initiated()

    print(x_train.shape)
    print(x_test.shape)
    print(y_train.shape)
    print(y_test.shape)

    model_trainer=Model_trainer(config=trainer_model,data=data,transformation=transformation)

    model_trainer.get_trained_model(x_train,x_test,y_train,y_test)

except Exception as e:
    raise e


[2023-08-21 20:52:58,332: INFO: common: safe loaded config\config.yaml]
[2023-08-21 20:52:58,336: INFO: common: safe loaded params.yaml]
[2023-08-21 20:52:58,339: INFO: common: The directory created]
[2023-08-21 20:52:58,341: INFO: common: The directory created]
[2023-08-21 20:52:58,603: INFO: data_transformation: creation of preprocessor_obj]
(72109, 11)
(24037, 11)
(72109,)
(24037,)
[('logistic_model', 0.6236303001429252)]
